In [4]:
import torch
from torch import nn
from torch.nn import functional as F

In [5]:
enc_features = torch.randn(1, 32, 64, 64)
dec_features = torch.randn(1, 32, 64, 64)

In [6]:
class Spatial_Attention(nn.Module):
    def __init__(self, en_in_channels, dec_in_channels):
        super().__init__()
        
        self.avg_pool = nn.AvgPool2d(kernel_size=64)
        self.max_pool = nn.MaxPool2d(kernel_size=64)
        # self.squeez_max = max_pool.squeeze(-1)
        self.con_1x1_enc = nn.Conv1d(en_in_channels, out_channels=2, kernel_size=1)
        self.con_1x1_dec = nn.Conv1d(dec_in_channels, out_channels=2, kernel_size=1)
        
        
        self.conv_1x1_after_add = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=1)
        # self.squeez_avg = avg_pool.squeeze(-1)
        # self.conv_1x1_avg = nn.Conv1d(dec_in_channels, out_channels=2, kernel_size=1)
        
        
        
    def forward(self, enc_features, dec_features):
        enc_avg = self.avg_pool(enc_features)
        enc_max = self.max_pool(enc_features)
        dec_avg = self.avg_pool(dec_features)
        dec_max = self.max_pool(dec_features)
        enc_squeez_avg = enc_max.squeeze(-1)
        enc_squeez_max = enc_avg.squeeze(-1)
        dec_squeez_avg = dec_max.squeeze(-1)
        dec_squeez_max = dec_avg.squeeze(-1)
        enc_con_1x1_avg = self.con_1x1_enc(enc_squeez_avg)
        enc_con_1x1_max = self.con_1x1_enc(enc_squeez_max)
        dec_con_1x1_avg = self.con_1x1_dec(dec_squeez_avg)
        dec_con_1x1_max = self.con_1x1_dec(dec_squeez_max)

        enc_add_max_avg = enc_con_1x1_avg + enc_con_1x1_max
        dec_add_max_avg = dec_con_1x1_avg + dec_con_1x1_max
        add_enc_dec = enc_add_max_avg + dec_add_max_avg
        
        # conv_1x1_after_add = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=1)
        conv_output = self.conv_1x1_after_add(add_enc_dec)
        
        un_squeez_dim = conv_output.unsqueeze(-1)
        un_squeez_dim.shape
        
        final_output = torch.sigmoid(un_squeez_dim)
        print("Final output shape after sigmoid activation:", final_output.shape)

    
        return final_output

In [7]:
if __name__ == "__main__":
    enc_features = torch.randn(1, 32, 64, 64)
    dec_features = torch.randn(1, 32, 64, 64)
    
    model = Spatial_Attention(en_in_channels=32, dec_in_channels=32)
    output = model(enc_features, dec_features)
    print("Output shape:", output.shape)

Final output shape after sigmoid activation: torch.Size([1, 32, 1, 1])
Output shape: torch.Size([1, 32, 1, 1])
